In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# At Intact, this session is usually already created for you in a notebook
spark = SparkSession.builder.appName("IntactInternship").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/03 13:39:59 WARN Utils: Your hostname, aduu-ThinkPad-P14s-Gen-4, resolves to a loopback address: 127.0.1.1; using 192.168.1.10 instead (on interface wlp0s20f3)
26/01/03 13:39:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/03 13:39:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/01/03 13:40:00 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
type(spark)

pyspark.sql.session.SparkSession

In [3]:
data = [
    (1, "Auto", 5000),
    (2, "Home", 12000),
    (3, "Auto", 3000),
    (4, "Life", 50000),
    (5, "Home", 8000)
]

columns = ["claim_id", "policy_type", "claim_amount"]

# Creating the DataFrame
df = spark.createDataFrame(data, columns)

In [4]:
type(df)

pyspark.sql.classic.dataframe.DataFrame

In [7]:
results = df.groupBy("policy_type").count()
print(results.show())

[Stage 0:>                                                        (0 + 16) / 16]

+-----------+-----+
|policy_type|count|
+-----------+-----+
|       Auto|    2|
|       Home|    2|
|       Life|    1|
+-----------+-----+

None


In [8]:
results = df.groupBy("policy_type").agg(F.sum("claim_amount"))
print(results.show())

+-----------+-----------------+
|policy_type|sum(claim_amount)|
+-----------+-----------------+
|       Auto|             8000|
|       Home|            20000|
|       Life|            50000|
+-----------+-----------------+

None


In [15]:
from pyspark.sql.functions import col
results =df.filter(col("claim_amount")>10000)
print(results.show())

+--------+-----------+------------+
|claim_id|policy_type|claim_amount|
+--------+-----------+------------+
|       2|       Home|       12000|
|       4|       Life|       50000|
+--------+-----------+------------+

None


### JOINS

In [16]:
# Claims data
claims_data = [
    (101, 5000), 
    (102, 12000), 
    (103, 3000)
]
claims_df = spark.createDataFrame(claims_data, ["policy_id", "claim_amount"])

# Policy data
policy_data = [
    (101, "Auto", "Aditya"), 
    (102, "Home", "Suresh"), 
    (104, "Life", "Priya") # Note: Policy 104 has no claims
]
policies_df = spark.createDataFrame(policy_data, ["policy_id", "policy_type", "customer_name"])

In [19]:
innerJoinedDf = claims_df.join(policies_df, on="policy_id") # default is inner join: how="inner"
print(innerJoinedDf.show())

+---------+------------+-----------+-------------+
|policy_id|claim_amount|policy_type|customer_name|
+---------+------------+-----------+-------------+
|      101|        5000|       Auto|       Aditya|
|      102|       12000|       Home|       Suresh|
+---------+------------+-----------+-------------+

None


In [23]:
leftJoinedDf = claims_df.join(policies_df, on="policy_id", how="left")
print(f"Left Joined Dataframe containing NULL Values: {leftJoinedDf.show()}")

+---------+------------+-----------+-------------+
|policy_id|claim_amount|policy_type|customer_name|
+---------+------------+-----------+-------------+
|      101|        5000|       Auto|       Aditya|
|      102|       12000|       Home|       Suresh|
|      103|        3000|       NULL|         NULL|
+---------+------------+-----------+-------------+

Left Joined Dataframe containing NULL Values: None


In [27]:

leftJoinedDf.fillna("Not Available", subset = ['customer_name'])
print(f"Updated to show NA: {leftJoinedDf.show()}")

+---------+------------+-----------+-------------+
|policy_id|claim_amount|policy_type|customer_name|
+---------+------------+-----------+-------------+
|      101|        5000|       Auto|       Aditya|
|      102|       12000|       Home|       Suresh|
|      103|        3000|       NULL|         NULL|
+---------+------------+-----------+-------------+

Updated to show NA: None
